# Intro

## Standard modules

In [1]:
import os, sys, json
import numpy as np

In [2]:
from tqdm.auto import tqdm, trange

In [3]:
from bicm import BipartiteGraph as BiG

## Hand-made modules

In [4]:
from melt import melt

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sarawalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sarawalk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Folders

In [5]:
TEXT_FOLDER='./NewProcessedData/texts/'

## Files

In [6]:
text_files=[file for file in os.listdir(TEXT_FOLDER) if file.endswith('.txt')]

In [7]:
text_files.sort()

In [8]:
text_files[0][:4]

'2015'

# Grab all texts

In [9]:
all_texts={}
for text_file in tqdm(text_files):
    year=text_file[:4]
    with open(TEXT_FOLDER+text_file, 'r') as f:
        _text=f.readlines()
    
    if len(_text)>1:
        _text=' '.join(_text)
    elif len(_text)==1:
        _text=_text[0]
    else:
        print(text_file)
    
    if len(_text)>0:
        if year not in all_texts.keys():
            all_texts[year]={}
            all_texts[year]['firms']=[]
            all_texts[year]['texts']=[]
        all_texts[year]['texts'].append(_text)
        all_texts[year]['firms'].append(text_file[8:-9])

  0%|          | 0/574 [00:00<?, ?it/s]

To be checked

# Operate with metode

## Dependencies

In [16]:
import os, sys, json
import numpy as np

from tqdm.auto import tqdm, trange

from bicm import BipartiteGraph as BiG

import string

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem.snowball import SnowballStemmer

from collections import Counter

from scipy.stats import geom

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sarawalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sarawalk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Class

In [17]:
class metode:
    
    
    def __init__(self, texts, row_names=None, alpha=0.01, lang=None):
        # biadjacency list
        self.texts=texts
        # row names
        if row_names is not None:
            self.row_names=row_names
        else:
            self.row_names=np.arange(len(row_names))
        # significance threshold
        self.alpha=alpha
        assert alpha<1 and alpha>0
        # language
        if lang is None:
            self.lang="english"
        else:
            # check that english is among the accepted languages by nltk
            self.lang=lang
        # get the stemmer
        self.stemmer = SnowballStemmer(self.lang, ignore_stopwords=True)
        
        # bad characters
        self.stop_words = list(stopwords.words(self.lang))
        self.bad_char=['©', '–', '‘', '’', '“', '”', "''", "'s",'``']
        self.super_bad_char=["'", "\\", "/", '+', "^^", ":", '£', '$']
        
        # get the biadjacency list
        #self.get_bili()
        # get the biadjacency matrix to feed bicm
        #self.get_all_tokens()
        #self.bili2bima()
    
    def _tests(self, entry):
        # I am removing:
        # - stop words;
        # - punctuation
        # - fractional numbers
        _test_0=not (entry in self.stop_words)
        _test_1=not (entry in self.bad_char)
        _test_2=not (entry in string.punctuation)
        #_test_3=not ('.' in entry)
        #_test_4=not (',' in entry)
        #_test_5=not entry.isnumeric()
        _test_5=not entry[0].isnumeric()
        _test_6=not (entry[0] in self.super_bad_char)
        #return _test_0 and _test_1 and _test_2 and _test_3 and _test_4 and _test_5# and _test_6
        return _test_0 and _test_1 and _test_2 and _test_5# and _test_6
    
    def get_bili(self):
        self.bili={}
        for i in trange(len(self.texts), leave=True, desc='get biadjacency list'):
            self.bili[self.row_names[i]]=self.text2tokens_counter(self.texts[i])
            
            
    def text2tokens_counter(self, text):
        # it could be parallelized
        tokens = [self.stemmer.stem(w.lower()) for w in word_tokenize(text) if self._tests(w)]
        return Counter(tokens)
    
    
    def get_all_tokens(self):
        self.all_tokens=[]
        for key in tqdm(self.bili.keys(), leave=True, desc='get all tokens'):
            for _token in self.bili[key].keys():
                if _token not in self.all_tokens:
                    self.all_tokens.append(_token)
        self.all_tokens.sort()
        self.all_tokens=np.array(self.all_tokens)

    def bili2bima(self):
        ''' 
        biadjacency list to biadjacency matrix
        '''
        if not hasattr(self, 'bili'):
            self.get_bili()
        if not hasattr(self, 'all_tokens'):
            self.get_all_tokens()
        
        self.bima=np.zeros((len(self.bili.keys()), len(self.all_tokens)), dtype=int)
        for i_key, key in enumerate(tqdm(self.bili.keys(), leave=True, desc='bili2bima')):
            for _token in self.all_tokens:
                where_token=np.where(self.all_tokens==_token)[0][0]
                #where_token=self.all_tokens.index(_token)
                self.bima[i_key, where_token]+=self.bili[key][_token]
                
    def validated_bima(self):
        '''
        Maximum Entropy TOpic DEtection
        '''
        if not hasattr(self, 'bima'):
            self.bili2bima()
            
        
        # Bipartite Graph of bicm
        self.mygraph=BiG()
        # initialize it with the bipartite matrix, 
        # i.e. the only way to initialize the method for bipartite weighted graphs
        self.mygraph.set_biadjacency_matrix(self.bima)
        # solve BiWCM
        self.mygraph.solve_tool()
        # calculate p-values
        self.mygraph.compute_weighted_pvals_mat()
        # return the validated matrix
        self.validated_m=self.mygraph.get_validated_matrix(significance=self.alpha, validation_method='fdr')
        
    def get_p_mat(self):
        if not hasattr(self, 'mygraph'):
            self.validated_bima()
        self.p_mat=np.outer(np.exp(-self.mygraph.theta_x), np.exp(-self.mygraph.theta_y))
        
    def sample_me(self):
        if not hasattr(self, 'p_mat'):
            self.get_p_mat()
        return geom.rvs(p_mat)-1
    
    def sufficient_sample(self, layer):
        '''
        The idea is to have a sample that should be big enough 
        to validate something with using Bonferroni
        '''
        _couples=int(l_ltp*(l_ltp-1)/2) # that's the number of test
        SAFETY_FACTOR=2. # maybe a little bit constipated
        return round(_couples*SAFETY_FACTOR/self.alpha)
        
        
    def p_val_covol(self, n_sample, layer):
        if not hasattr(self, 'p_mat'):
            self.covol={}
        
        l_ltp=l_ltp=self.bima.shape[layer]
        # sampling covolumes from biwcm
        covol_sample=np.zeros((int(l_ltp*(l_ltp-1)/2), n_sample), dtype=int)
        # it could be parallelized
        for _ in trange(n_sample, leave=True):
            ran_mat=self.sample_me()
            counter=0
            for i in range(l_ltp):
                for j in range(i+1, l_ltp):
                    covol_sample[counter, _]=np.dot(ran_mat[i], ran_mat[j])
                    counter+=1
        self.covol[layer]={}
        self.covol[layer]['sample']=covol_sample
        
        # observed covolumes            
        self.covol_obs=np.zeros(int(l_ltp*(l_ltp-1)/2), dtype=int)
        counter=0
        for i in range(l_ltp):
            for j in range(i+1, l_ltp):
                covol_obs[counter]=np.dot(debug.bima[i], debug.bima[j])
                counter+=1
        self.covol[layer]['obs']=covol_obs
        
        # p-value
        p_val=np.zeros(l_ltp)
        for i in range(l_ltp):
            p_val[i]=np.sum(covol_sample[i, :]>=covol_obs[i])/n_sample
        self.covol[layer]['pval']=p_val

## Debug

In [12]:
debug=metode(all_texts['2023']['texts'][:10], all_texts['2023']['firms'][:10], 0.05)

In [13]:
debug.get_bili()

get biadjacency list:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
debug.validated_bima()

get all tokens:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Discrete weighted model: BiWCM_d


/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:326: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:327: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


max rows error = 1.1557509347336059e-05
max columns error = 1.7915442113252258e-06
total error = 0.0008624186946751949
Solver converged.


### Sample me!

In [26]:
from scipy.stats import geom

In [76]:
layer_to_project_to=1

In [77]:
l_ltp=debug.bima.shape[layer_to_project_to]

In [78]:
n_sample=2*round(l_ltp**2/debug.alpha)

In [80]:
f'{n_sample:,}'

'2,734,392,960'

In [85]:
np.all(np.exp(-debug.mygraph.theta_x)==debug.mygraph.x)

True

In [86]:
np.all(np.exp(-debug.mygraph.theta_y)==debug.mygraph.y)

True

In [33]:
p_mat=np.outer(np.exp(-debug.mygraph.theta_x), np.exp(-debug.mygraph.theta_y))

In [37]:
p_mat.shape==debug.bima.shape

True

In [41]:
ran_mat=geom.rvs(p_mat)-1

In [50]:
covol_sample=np.zeros((int(l_ltp*(l_ltp-1)/2), n_sample), dtype=int)
for _ in trange(n_sample, leave=True):
    ran_mat=geom.rvs(p_mat)-1
    counter=0
    for i in range(l_ltp):
        for j in range(i+1, l_ltp):
            covol_sample[counter, _]=np.dot(ran_mat[i], ran_mat[j])
            counter+=1

  0%|          | 0/4000 [00:00<?, ?it/s]

In [53]:
covol_obs=np.zeros(int(l_ltp*(l_ltp-1)/2), dtype=int)
counter=0
for i in range(l_ltp):
    for j in range(i+1, l_ltp):
        covol_obs[counter]=np.dot(debug.bima[i], debug.bima[j])
        counter+=1

In [67]:
p_val=np.zeros(l_ltp)
for i in range(l_ltp):
    p_val[i]=np.sum(covol_sample[i, :]>=covol_obs[i])/n_sample

In [68]:
p_val

array([0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.04225,
       0.     , 0.     , 1.     ])

In [69]:
covol_sample[0, :]

array([395113, 415166, 414210, ..., 382982, 403303, 399563])

In [71]:
covol_obs[0]

454166

In [75]:
debug.alpha

0.05

In [74]:
np.sort(p_val)<=(np.arange(len(p_val))+1)/len(p_val)*debug.alpha

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False])

# Melt me!

## Binary

In [11]:
all_texts.keys()

dict_keys(['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'])

In [12]:
[(key, len(all_texts[key]['firms'])) for key in all_texts.keys()]

[('2015', 40),
 ('2016', 44),
 ('2017', 51),
 ('2018', 56),
 ('2019', 67),
 ('2020', 69),
 ('2021', 76),
 ('2022', 80),
 ('2023', 89),
 ('2024', 2)]

**BINARY**

In [13]:
cacca=melt(all_texts['2023']['texts'], binary=True)

  0%|          | 0/89 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
cacca.get_projection(rows=True, alpha=0.05, approx_method='poisson', threads_num=4, progress_bar=True)


                       of the opposite layer. This may cause some convergence issues.
                      Please use the full mode providing a biadjacency matrix or an edgelist,
                       or clean your data from these nodes. 
                      


/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:326: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:327: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


max rows error = 4.547473508864641e-12
max columns error = 9.947598300641403e-14
total error = 5.441136430306415e-11
Solver converged.


  7%|████████▏                                                                                                                | 6/89 [00:00<00:09,  8.66it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered in divide
  probs = node_xy * neighbor_xy / ((1 + node_xy) * (1 + neighbor_xy))
 13%|████████████████▏                                                                                                       | 12/89 [00:01<00:08,  9.04it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered in divide
  probs = node_xy * neighbor_xy / ((1 + node_xy) * (1 + neighbor_xy))
 20%|████████████████████████▎                                                                                               | 18/89 [00:01<00:07,  9.17it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered 

No V-motifs will be validated. Try increasing alpha


AttributeError: 'BipartiteGraph' object has no attribute 'cols_projection'

In [14]:
cacca.get_projection(rows=False, alpha=0.05, approx_method='poisson', threads_num=4, progress_bar=True)

 13%|██████████████▎                                                                                                   | 3884/31060 [00:16<01:54, 237.82it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered in divide
  probs = node_xy * neighbor_xy / ((1 + node_xy) * (1 + neighbor_xy))
 19%|█████████████████████▍                                                                                            | 5826/31060 [00:24<01:40, 251.41it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered in divide
  probs = node_xy * neighbor_xy / ((1 + node_xy) * (1 + neighbor_xy))
 25%|████████████████████████████▌                                                                                     | 7768/31060 [00:30<01:28, 264.35it/s]/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/graph_classes.py:1113: RuntimeWarning: invalid value encountered 

No V-motifs will be validated. Try increasing alpha


AttributeError: 'BipartiteGraph' object has no attribute 'cols_projection'

In [ ]:
cacca.token_proj

Not only it doesn't work, but it also takes a lot of time.

## Weighted

bicm reads only weighted biadjacency matrices, therefore...

### Function

In [29]:
# biadjacency list to biadjacency matrix
def bili2bima(bili):
    # get the number of different tokens
    all_tokens=[]
    for key in tqdm(bili.keys(), leave=True):
        for _token in bili[key].keys():
            if _token not in all_tokens:
                all_tokens.append(_token)
    # transform all_tokens to a numpy array, such that I can use np.unique safely
    all_tokens=np.array(all_tokens)
    # define the biadjacency matrix 
    bima=np.zeros((len(bili.keys()), len(all_tokens)), dtype=int)
    for key in tqdm(bili.keys(), leave=True):
        for _token in bili[key].keys():
            where_token=np.where(all_tokens==_token)[0][0]
            bima[key, where_token]+=bili[key][token]
    return bima, all_tokens

### Debug

In [30]:
len(all_texts['2023']['texts'])

89

In [44]:
cacca=melt(all_texts['2023']['texts'], binary=False)

  0%|          | 0/89 [00:00<?, ?it/s]

In [162]:
len(cacca.biadj_list)

89

In [28]:
aux, aux_at=bili2bima(cacca.biadj_list)

NameError: name 'bili2bima' is not defined

In [30]:
bili=cacca.biadj_list.copy()

In [101]:
bad_char=['-', '©', '–', '‘', '’', '“', '”', "''", "'s",'``', '\\', '|']

In [26]:
    all_tokens=[]
    for key in tqdm(bili.keys(), leave=True):
        for _token in bili[key].keys():
            if _token not in all_tokens and all([bd not in _token for bd in bad_char]) and _token[0] not in ["'", "\\", "/", '+', "^^", ":", '£', '$'] and not _token[0].isnumeric():
                all_tokens.append(_token)
    all_tokens=np.array(all_tokens)

NameError: name 'bili' is not defined

In [27]:
    bima=np.zeros((len(bili.keys()), len(all_tokens)), dtype=int)
    for key in tqdm(bili.keys(), leave=True):
        for _token in bili[key].keys():
            if _token in all_tokens:
                where_token=np.where(all_tokens==_token)[0][0]
                bima[key, where_token]+=bili[key][_token]
    

NameError: name 'bili' is not defined

In [113]:
cacca=BiG()

In [114]:
cacca.set_biadjacency_matrix(bima)

Discrete weighted model: BiWCM_d


In [115]:
cacca.solve_tool()

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:326: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:327: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


max rows error = 0.00010472287903845316
max columns error = 7.311720381864007e-06
total error = 0.00773124083942919
Solver converged.


In [116]:
cacca.compute_weighted_pvals_mat()

In [117]:
cacca.pvals_mat

array([[1.45918023e-01, 4.66682797e-05, 3.74940711e-02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 2.41590737e-03, 3.69701886e-02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.18151329e-02, 5.45686665e-04, 1.18151329e-02]])

In [119]:
pval_m=cacca.get_validated_matrix(significance=0.05, validation_method='fdr')

In [126]:
for i in range(len(pval_m)):
    print(all_tokens[np.where(pval_m[i]==1)[0]], all_texts['2023']['firms'][i])

['africa' 'alto' 'american' 'anglo' 'barro' 'beer' 'berlin' 'botswana'
 'bronc' 'capcoal' 'carcinogen' 'chile' 'closur' 'copper' 'cpr' 'crd'
 'cybersecur' 'dam' 'dewat' 'diamond' 'edna' 'el' 'envusa' 'fcev'
 'futuresmart' 'gbv' 'gistm' 'hds' 'heritag' 'hiv' 'host' 'ibi' 'icmm'
 'irma' 'iron' 'kolomela' 'kumba' 'learn+' 'learner' 'los' 'mina' 'mine'
 'miner' 'mining™' 'mitsubishi' 'mogalakwena' 'moquegua' 'mt' 'nickel'
 'nois' 'npi' 'nutrient' 'oel' 'ore' 'peru' 'pgms' 'pionero' 'pmlu'
 'polyhalit' 'psm' 'queensland' 'quellaveco' 'rehabilit' 'rescu' 'resettl'
 'rjc' 'rock' 'rvms' 'sishen' 'slp' 'smp' 'soldado' 'south' 'southern'
 'spatial' 'steelmak' 'tail' 'teacher' 'tsfs' 'underground' 'unki'
 'valutrax™' 'vam' 'vfl' 'violenc' 'woodsmith' 'yourvoic' 'zimbabw'
 'zimel'] ANGLO_AMERICAN_PLC
['impact' 'ingredi' 'sustain' 'croda'] CRODA_INTERNATIONAL_PLC
['endeavour' 'gold' 'hectar' 'malaria' 'mine' 'pit' 'reforest' 'rehabilit'
 'resettl' 'tsf' 'villag' 'asgm' 'boungou' 'burkina' 'cyanid' 

In [124]:
all_texts['2023']['firms'][0]

'ANGLO_AMERICAN_PLC'

In [128]:
np.where(all_tokens=='sdg')

(array([2737]),)

In [133]:
all_tokens[2737:2740]

array(['sdg', 'sdg16', 'sdgs'], dtype='<U90')

In [138]:
np.sum(pval_m, axis=0)[2737:2740]

array([1, 0, 0], dtype=uint64)

In [141]:
all_texts['2023']['firms'][np.where(pval_m[:, 2737]==1)[0][0]]

'RELX_PLC'

In [143]:
np.any(np.sum(pval_m, axis=1)==0)

False

### Maximum Entropy TOpic DEtection (METODE)

In [12]:
import string

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sarawalk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sarawalk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from collections import Counter

In [14]:
from collections import defaultdict

In [15]:
from scipy.stats import geom

In [16]:
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
class metode:
    
    
    def __init__(self, texts, row_names=None, alpha=0.01, lang=None, parallel=False):
        # biadjacency list
        self.texts=texts
        # row names
        if row_names is not None:
            self.row_names=np.array(row_names)
        else:
            self.row_names=np.arange(len(texts))
        # significance threshold
        self.alpha=alpha
        assert alpha<1 and alpha>0
        # language
        if lang is None:
            self.lang="english"
        else:
            # check that english is among the accepted languages by nltk
            self.lang=lang
            
        self.parallel=parallel
            
        # get the stemmer
        self.stemmer = SnowballStemmer(self.lang, ignore_stopwords=True)
        
        # bad characters
        self.stop_words = list(stopwords.words(self.lang))
        self.bad_char=['©', '–', '‘', '’', '“', '”', "''", "'s",'``', '--', '|', '..']
        #self.super_bad_char=["'", "\\", "/", '+', "^^", ":", '£', '$']
        self.super_bad_char=["'", "\\", "/", '+', "^^", ":", '£', '$', '€', '~', '≥', '-']
        
        # get the biadjacency list
        #self.get_bili()
        # get the biadjacency matrix to feed bicm
        #self.get_all_tokens()
        #self.bili2bima()
    
    def _tests(self, entry):
        # I am removing:
        # - stop words;
        # - punctuation
        # - fractional numbers
        _test_0=not (entry in self.stop_words)
        _test_1=not (entry in self.bad_char)
        _test_2=not (entry in string.punctuation)
        _test_3=not (entry[0] in string.punctuation)
        _test_3=not ('.' in entry)
        _test_4=not (',' in entry)
        _test_5=not entry.isnumeric()
        _test_6=not entry[0].isnumeric()
        _test_7=not (entry[0] in self.super_bad_char)
        _test_8=not ('|' in entry)
        #return _test_0 and _test_1 and _test_2 and _test_3 and _test_4 and _test_5# and _test_6
        return _test_0 and _test_1 and _test_2 and _test_5 and _test_6 and _test_7 and len(entry)>0
    
    def get_bili(self):
        self.bili={}
        for i in trange(len(self.texts), leave=False, desc='get biadjacency list'):
            self.bili[self.row_names[i]]=self.text2tokens(self.texts[i])
            
    def text2tokens(self, text):
        tokens = [self.stemmer.stem(w.lower()) for w in word_tokenize(text) if self._tests(w)]
        return Counter(tokens)
    
    def get_all_tokens(self):
        self.all_tokens=[]
        for key in tqdm(self.bili.keys(), leave=False, desc='get all tokens'):
            for _token in self.bili[key].keys():
                if _token not in self.all_tokens:
                    self.all_tokens.append(_token)
        self.all_tokens.sort()
        self.all_tokens=np.array(self.all_tokens)

    def get_bima(self):
        ''' 
        biadjacency list to biadjacency matrix
        '''
        if not hasattr(self, 'bili'):
            self.get_bili()
        if not hasattr(self, 'all_tokens'):
            self.get_all_tokens()
            
        self.bima=np.zeros((len(self.bili.keys()), len(self.all_tokens)), dtype=int)
        for i_key, key in enumerate(tqdm(self.bili.keys(), leave=False)):
            for _token in self.all_tokens:
                where_token=np.where(self.all_tokens==_token)[0][0]
                #where_token=self.all_tokens.index(_token)
                self.bima[i_key, where_token]+=self.bili[key][_token]
                
    def get_biwcm(self):
        if not hasattr(self, 'bima'):
            self.get_bima()
        # Bipartite Graph of bicm
        self.mygraph=BiG()
        # initialize it with the bipartite matrix, 
        # i.e. the only way to initialize the method for bipartite weighted graphs
        self.mygraph.set_biadjacency_matrix(self.bima)
        # solve BiWCM
        self.mygraph.solve_tool()
        # probability for the geometric distribution (to feed numpy function)
        self.p_mat=np.outer(self.mygraph.x, self.mygraph.y)
        self.p_mat=1-self.p_mat
        
        
    def get_validated_bima(self):
        if not hasattr(self, 'mygraph'):
            self.get_biwcm()
        # calculate p-values    
        self.mygraph.compute_weighted_pvals_mat()
        self.val_bima=self.mygraph.get_validated_matrix(significance=self.alpha, validation_method='fdr')
        
    def sample_me(self):
        rng = np.random.default_rng()
        return rng.geometric(self.p_mat)+1
    
    def fdr_suf_sam(self, layer):
        '''
        FDR sufficient sample
        '''
        if layer==0:
            l_layer=len(self.texts)
        else:
            if not hasattr(self, 'all_tokens'):
                self.get_all_tokens()
            l_layer=len(self.all_tokens)
        n_couples=int(l_layer*(l_layer-1)/2)
        SAFETY_FACTOR=2. # quite constipated, but it's ok
        return round(n_couples*SAFETY_FACTOR/self.alpha)
    
    @staticmethod
    def covol_calculator(mat, layer):
        if layer==0:
            aux_mat=mat.copy()
        else:
            aux_mat=mat.T.copy()
        l_layer=mat.shape[layer]
        n_couples=int(l_layer*(l_layer-1)/2)
        covol=np.zeros(n_couples, dtype=int)
        counter=0
        for i in range(l_layer):
            for j in range(i+1, l_layer):
                covol[counter]=np.dot(aux_mat[i], aux_mat[j])
                counter+=1
        return covol
    
    def get_validated_covol(self, layer, n_sample):
        # let's first get the empirical value...
        if not hasattr(self, 'bima'):
            self.get_bima()
        
        covol_obs=self.covol_calculator(self.bima, layer)
        
        # ... then the sampled
        if not hasattr(self, 'mygraph'):
            self.get_biwcm()
        
        covol_sample=np.zeros((len(covol_obs), n_sample))
        
        if self.parallel:
            results = []
            with ProcessPoolExecutor() as executor:
                futures = [executor.submit(self.covol_calculator, self.sample_me(), layer) for _ in range(n_sample)]
            for future in tqdm(as_completed(futures), total=n_sample, desc="Processing samples"):
                results.append(future.result())
    
            # Costruisci l'array risultante assumendo che ogni risultato sia un vettore
            covol_sample = np.column_stack(results)
            
            
        else:
            for _ in trange(n_sample, leave=True):
                covol_sample[:, _]=self.covol_calculator(self.sample_me(), layer)        
        
        # finally, let's calculate the p-values...
        self.p_vals=np.zeros(len(covol_obs), dtype=[('node_0','i4'), ('node_1','i4'), ('p_value','f8')])
        l_layer=self.bima.shape[layer]
        
        counter=0
        for i in range(l_layer):
            for j in range(i+1, l_layer):
                self.p_vals[counter]['node_0']=i
                self.p_vals[counter]['node_1']=j
                self.p_vals[counter]['p_value']=np.sum(covol_sample[counter]>=covol_obs[counter])/n_sample
                counter+=1
                
        #... and validate them
        self.effective_th=self.fdr_th(self.p_vals['p_value'], self.alpha)
        self.val_covol=self.p_vals[self.p_vals['p_value']<=self.effective_th][['node_0', 'node_1']]
        
     
    @staticmethod
    def fdr_th(p_vals, alpha):
        n_p_vals=len(p_vals)
        s_p_vals=np.sort(p_vals)
        running_bonf=(1+np.arange(n_p_vals))*alpha/n_p_vals
        _satisfaction=s_p_vals<=running_bonf
        if np.sum(_satisfaction)>0:
            # there at least a p-value satisfying the condition
            return running_bonf[_satisfaction][-1]
        else:
            # there are no p-values satisfying the condition
            return 0

### Debug

In [151]:
debug=metode(all_texts['2023']['texts'][:10], all_texts['2023']['firms'][:10], 0.01)

In [61]:
debug.get_bili()

get biadjacency list:   0%|          | 0/10 [00:00<?, ?it/s]

In [62]:
debug.get_all_tokens()

get all tokens:   0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
len(debug.all_tokens)

8677

In [64]:
debug.all_tokens

array(['', ',ywell-b', '--', ..., '≥', '≥10', '≥oel'], dtype='<U94')

In [66]:
debug.all_tokens[40]

'1.1'

In [65]:
debug._tests(debug.all_tokens[40])

False

In [161]:
_cacca=word_tokenize(all_texts['2023']['texts'][1])

In [162]:
_cacca.sort()

In [163]:
_cacca=np.unique(_cacca)

In [164]:
_cacca

array(['%', '&', "'", ..., '”', '•', '∆'], dtype='<U73')

In [165]:
tokens = [debug.stemmer.stem(w.lower()) for w in np.unique(_cacca) if debug._tests(w)]

In [166]:
tokens.sort()

In [167]:
len(tokens)

2054

In [168]:
len(np.unique(tokens))

1398

In [22]:
debug.get_bima()

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
debug.get_biwcm()

Discrete weighted model: BiWCM_d


/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:326: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/bicm/models_functions.py:327: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/numba/core/utils.py:643: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


max rows error = 1.1557509347336059e-05
max columns error = 1.7915442113252258e-06
total error = 0.0008624186946751949
Solver converged.


#### Debug of p-value calculator

In [37]:
debug.get_validated_bima()

In [38]:
debug.fdr_suf_sam(0)

9000

In [73]:
n_sample=5*int(debug.fdr_suf_sam(0))

In [74]:
debug.parallel=False

In [75]:
debug.get_validated_covol(0, n_sample)

  0%|          | 0/45000 [00:00<?, ?it/s]

In [76]:
unparallel_pvals=debug.p_vals.copy()

In [93]:
debug.parallel=True

In [78]:
debug.get_validated_covol(0, n_sample)

Process ForkProcess-48:
Process ForkProcess-44:
Process ForkProcess-43:
Process ForkProcess-47:
Process ForkProcess-38:
Process ForkProcess-39:
Process ForkProcess-42:
Process ForkProcess-33:
Process ForkProcess-31:
Process ForkProcess-40:
Process ForkProcess-41:
Process ForkProcess-46:
Process ForkProcess-34:
Process ForkProcess-36:
Process ForkProcess-45:
Process ForkProcess-37:
Process ForkProcess-35:


KeyboardInterrupt: 

Process ForkProcess-32:
Process ForkProcess-30:
Process ForkProcess-28:
Process ForkProcess-29:
Process ForkProcess-25:
Process ForkProcess-26:
Process ForkProcess-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessin

  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/sarawalk/python/Python-3.8.10/Lib/concurrent/futures/process.py", line 233

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessing/queues.py", line 97, in get
    res = self._recv_bytes()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/home/sarawalk/python/Python-3.8.10/Lib/multiprocessing/connecti

In [ ]:
parallel_pvals=debug.p_vals.copy()

In [79]:
np.vstack((np.abs(parallel_pvals['p_value']-unparallel_pvals['p_value'])/unparallel_pvals['p_value'], unparallel_pvals['p_value'])).T

/tmp/ipykernel_14696/1325511895.py:1: RuntimeWarning: invalid value encountered in divide
  np.vstack((np.abs(parallel_pvals['p_value']-unparallel_pvals['p_value'])/unparallel_pvals['p_value'], unparallel_pvals['p_value'])).T


array([[5.10510511e-02, 7.40000000e-03],
       [1.66734154e-04, 9.32955556e-01],
       [9.11096193e-04, 9.26844444e-01],
       [6.66080762e-03, 4.80422222e-01],
       [5.16605166e-03, 4.51666667e-01],
       [4.44721160e-05, 9.99377778e-01],
       [1.90476190e-01, 1.40000000e-03],
       [3.48301737e-03, 9.50644444e-01],
       [1.56812339e-01, 8.64444444e-03],
       [9.75609756e-02, 9.11111111e-04],
       [           nan, 0.00000000e+00],
       [6.88405797e-03, 3.06666667e-01],
       [           nan, 0.00000000e+00],
       [           nan, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [           nan, 0.00000000e+00],
       [2.08583752e-03, 6.28577778e-01],
       [1.55444101e-02, 6.24733333e-01],
       [2.43309002e-03, 2.00933333e-01],
       [4.63857750e-03, 1.14977778e-01],
       [3.47649743e-03, 9.20466667e-01],
       [           nan, 0.00000000e+00],
       [8.17671989e-03, 5.19088889e-01],
       [2.33333333e+00, 6.66666667e-05],
       [1.073825

Non bene...

In [44]:
len(debug.p_vals)

45

In [31]:
f'{debug.fdr_suf_sam(1):,}'

'1,367,031,120'

#### ChatGPT's functions and approximations

In [24]:
from covolumes_pvals import p_value_dot_product_numba 

In [80]:
from covolumes_pval_approx import p_value_mix_numba

##### Test

In [26]:
covol_obs=debug.covol_calculator(debug.bima, 0)
covol_obs

array([ 454166, 1773244, 1281563,  742991, 1072173, 1888740,  445531,
       1435494,  521473,  189911,  181004,  100673,  150391,  257898,
         61333,  200151,   73980,  533529,  320214,  437739,  812252,
        191404,  612181,  221355,  290378,  485514,  776947,  159537,
        585934,  194995,  221541,  399251,   95059,  285545,  100341,
        655881,  137047,  492557,  157820,  267089,  829716,  274333,
        172382,   63076,  224623])

In [27]:
len(debug.all_tokens)

8268

In [91]:
p_value_dot_product_numba(1-debug.p_mat[0][:10], 1-debug.p_mat[1][:10], int(covol_obs[0]))

2.1474220117566175e-10

In [92]:
p_value_dot_product_numba(1-debug.p_mat[0][:100], 1-debug.p_mat[1][:100], int(covol_obs[0]))

1.633742655024922e-05

In [93]:
p_value_dot_product_numba(1-debug.p_mat[0][:200], 1-debug.p_mat[1][:200], int(covol_obs[0]))

4.0100410482547613e-07

In [94]:
p_value_dot_product_numba(1-debug.p_mat[0][:500], 1-debug.p_mat[1][:500], int(covol_obs[0]))

3.755430519635083e-06

In [28]:
p_value_mix_numba(1-debug.p_mat[0][:10], 1-debug.p_mat[1][:10], int(covol_obs[0]))

/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)
/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)


0.0

In [29]:
p_value_mix_numba(1-debug.p_mat[0][:100], 1-debug.p_mat[1][:100], int(covol_obs[0]))

/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)
/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)


0.0

In [30]:
p_value_mix_numba(1-debug.p_mat[0][:200], 1-debug.p_mat[1][:200], int(covol_obs[0]))

0.0

In [31]:
p_value_mix_numba(1-debug.p_mat[0][:500], 1-debug.p_mat[1][:500], int(covol_obs[0]))

0.0

In [36]:
f'{debug.alpha/45:.2e}'

'2.22e-04'

I mean, it could even work, since th difference from the real values are extremely low...

##### Check with sampling

In [81]:
chatgpt_p_vals=np.zeros(len(covol_obs), dtype=[('node_0','i4'), ('node_1','i4'), ('p_value','f8')])

In [82]:
counter=0
for i in trange(debug.bima.shape[0]):
    for j in range(i+1, debug.bima.shape[0]):
        chatgpt_p_vals[counter]['node_0']=i
        chatgpt_p_vals[counter]['node_1']=j
        # w=k-1 
        chatgpt_p_vals[counter]['p_value']=p_value_mix_numba(debug.p_mat[i], debug.p_mat[j], int(covol_obs[counter])+1)
        counter+=1

  0%|          | 0/10 [00:00<?, ?it/s]

/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)
/home/sarawalk/sdgs_py38/sdg_repos_1/covolumes_pval_approx.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - st.norm.cdf(w) + st.norm.pdf(w)*(1/w - 1/u)


In [88]:
np.vstack((chatgpt_p_vals['p_value'], parallel_pvals['p_value'], unparallel_pvals['p_value'])).T

array([[0.00000000e+00, 7.77777778e-03, 7.40000000e-03],
       [0.00000000e+00, 9.33111111e-01, 9.32955556e-01],
       [0.00000000e+00, 9.26000000e-01, 9.26844444e-01],
       [0.00000000e+00, 4.77222222e-01, 4.80422222e-01],
       [0.00000000e+00, 4.54000000e-01, 4.51666667e-01],
       [9.98500000e-01, 9.99333333e-01, 9.99377778e-01],
       [0.00000000e+00, 1.66666667e-03, 1.40000000e-03],
       [0.00000000e+00, 9.47333333e-01, 9.50644444e-01],
       [0.00000000e+00, 1.00000000e-02, 8.64444444e-03],
       [0.00000000e+00, 1.00000000e-03, 9.11111111e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.08777778e-01, 3.06666667e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.29888889e-01, 6.28577778e-01],
       [0.00000000e+00, 6.34444

In [89]:
eff_th=debug.fdr_th(debug.p_vals['p_value'], debug.alpha)

In [90]:
np.vstack((chatgpt_p_vals['p_value']<=eff_th, parallel_pvals['p_value']<=eff_th, unparallel_pvals['p_value']<=eff_th)).T

array([[ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [False, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True, False, False],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True,  True,  True],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ T

In [91]:
np.sum((parallel_pvals['p_value']<=eff_th)==(unparallel_pvals['p_value']<=eff_th))

44

In [92]:
np.sum((parallel_pvals['p_value']<=eff_th)!=(unparallel_pvals['p_value']<=eff_th))

1

### Let the children play

In [181]:
for year in tqdm(all_texts.keys()):
    if year!='2024':
        _cacca=metode(all_texts[year]['texts'], all_texts[year]['firms'], 0.01)
        _cacca.get_bima()
        np.savetxt('./ciruffa_files/'+year+'_companies.txt', all_texts[year]['firms'], delimiter=',', fmt='%s')
        np.savetxt('./ciruffa_files/'+year+'_tokens.txt', _cacca.all_tokens, delimiter=',', fmt='%s')
        np.savetxt('./ciruffa_files/'+year+'_bima.txt', _cacca.bima, delimiter=',', fmt='%i')

  0%|          | 0/10 [00:00<?, ?it/s]

get biadjacency list:   0%|          | 0/40 [00:00<?, ?it/s]

get all tokens:   0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

get biadjacency list:   0%|          | 0/44 [00:00<?, ?it/s]

...

get all tokens:   0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

get biadjacency list:   0%|          | 0/89 [00:00<?, ?it/s]

get all tokens:   0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]